# Test

In [1]:
import molsysmt as msm
from molsysmt.systems import tests as tests_systems

In [2]:
molsys = msm.convert(tests_systems['chicken villin HP35']['chicken_villin_HP35_solvated.msmpk'], to_form='molsysmt.MolSys')

In [3]:
msm.info(molsys)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_ions,n_peptides,n_structures
molsysmt.MolSys,5050,1521,1484,3,1484,1,1481,2,1,1
